<a href="https://colab.research.google.com/github/frank-morales2020/MLxDL/blob/main/MYVERSION_OnDemandLoaderTool.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# OnDemandLoaderTool

Our `OnDemandLoaderTool` is a powerful agent tool that allows for "on-demand" data querying from any data source on LlamaHub.

This tool takes in a `BaseReader` data loader, and when called will 1) load data, 2) index data, and 3) query the data.

In this walkthrough, we show how to use the `OnDemandLoaderTool` to convert our Wikipedia data loader into an accessible search tool for a LangChain agent.

In [ ]:
%pip install llama-index-readers-wikipedia -q

In [ ]:
!pip install llama-index -q

In [ ]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

In [ ]:
!pip install wikipedia -q

In [ ]:
reader = WikipediaReader()

In [ ]:
tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

In [ ]:
!pip install langchain -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
%pip install openai  --root-user-action=ignore -q
%pip install colab-env --upgrade --quiet --root-user-action=ignore -q

In [ ]:
#added by Frank Morales(FM) 22/02/2024
import warnings
warnings.filterwarnings('ignore')

import colab_env
import openai
import os
openai.api_key = os.getenv("OPENAI_API_KEY")

from openai import OpenAI
client = OpenAI()

In [ ]:
tool(["Montreal"], query_str="where is located the Montreal city?")

In [ ]:
# run tool as langchain structured tool
lc_tool = tool.to_langchain_structured_tool(verbose=True)

In [41]:
from langchain.agents import AgentType, initialize_agent, load_tools
#from langchain_openai import ChatOpenAI, OpenAI
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI

#llm = ChatOpenAI(temperature=0.0)
math_llm = OpenAI(temperature=0.0)

#llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)
llm = ChatOpenAI(temperature=0, model_name="gpt-4", streaming=True)

tools = load_tools(
    ["human", "llm-math"],
    llm=math_llm,
)

agent_chain = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    #agent="structured-chat-zero-shot-react-description",
    verbose=True,
)

agent = initialize_agent(
    [lc_tool],
    #tools,
    llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False,
)

In [42]:
agent_chain.run("What's my friend Rafael's surname?") ## GPT-3.5
# Answer with 'Reinaldo'



> Entering new AgentExecutor chain...
I need to ask a human for this information.
Action: human
Action Input: Can you tell me Rafael's surname?

Can you tell me Rafael's surname?
Reinaldo

Observation: Reinaldo
Thought:I now know Rafael's surname.
Final Answer: Rafael's surname is Reinaldo.

> Finished chain.


"Rafael's surname is Reinaldo."

In [ ]:
#agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"},query_str="What's the arts and culture scene in montreal?")

agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"}) ## GPT-4



Observation: Montreal has a rich and diverse arts and culture scene, with a strong emphasis on music, festivals, and performing arts. The city is known for its vibrant cultural life, with over 100 annual festivals including the Montreal International Jazz Festival, the largest jazz festival in the world, and the Just for Laughs comedy festival. Montreal is also home to the Place des Arts complex, which hosts a variety of classical music performances, theatre productions, and dance shows. The city's cultural heritage is reflected in its numerous churches, with Montreal being renowned for its many historic churches and basilicas. Additionally, Montreal has been a significant setting for Quebec literature, with writers capturing the essence of the multicultural city in their works.
Thought:

"Montreal has a rich and diverse arts and culture scene, with a strong emphasis on music, festivals, and performing arts. The city is known for its vibrant cultural life, with over 100 annual festivals including the Montreal International Jazz Festival, the largest jazz festival in the world, and the Just for Laughs comedy festival. Montreal is also home to the Place des Arts complex, which hosts a variety of classical music performances, theatre productions, and dance shows. The city's cultural heritage is reflected in its numerous churches, with Montreal being renowned for its many historic churches and basilicas. Additionally, Montreal has been a significant setting for Quebec literature, with writers capturing the essence of the multicultural city in their works."

In [ ]:
response=agent.run(input={"pages": "Montreal", "query_str": "What is the best restaurant in Montreal?"}) ## GPT-4

In [ ]:
response

"The best restaurant in Montreal can vary depending on personal preferences, as the city is known for its diverse culinary scene. From traditional French cuisine to trendy cafes and international flavors, there's something for everyone. It's recommended to explore online reviews, ask locals for recommendations, and consider the type of cuisine or dining experience you're seeking to find the best restaurant that suits your taste."

In [ ]:
!pip install llama_index -q
!pip install langchain -q
!pip install wikipedia -q

In [43]:
import os
import openai

import llama_index

#from llama_index.tools.ondemand_loader_tool import OnDemandLoaderTool
## added by frank morales
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool

from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

reader = WikipediaReader()

tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="Wikipedia Tool",
    description="A tool for loading and querying articles from Wikipedia",
)

# run only the llama_Index Tool by itself
#tool(["Cuba"], query_str="What's the arts and culture scene in Cuba?")

# run tool from LangChain Agent
lc_tool = tool.to_langchain_structured_tool(verbose=True)
#response=lc_tool.run(tool_input={"pages": ["Cuba"], "query_str": "What's the capital city of Cuba?"})
response=lc_tool.run(tool_input={"pages": ["Cuba"], "query_str": "What's the arts and culture scene in Cuba?"})

Cuba has a rich arts and culture scene with influences from various sources. The country is known for its vibrant music, including genres like son, salsa, and Afro-Cuban jazz. Cuban visual arts have a strong tradition, with artists like Wifredo Lam gaining international recognition. Cuban literature has produced notable figures such as José Martí and Alejo Carpentier. The country also has a strong tradition of dance, with the Cuban National Ballet being renowned worldwide. Overall, Cuba's arts and culture scene is diverse and reflects the country's unique history and heritage.

Initialise LangChain Agent

In [ ]:
from langchain.agents import initialize_agent
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(temperature=0, model_name="gpt-3.5-turbo", streaming=True)

agent = initialize_agent(
    [lc_tool],
    llm=llm,
    agent="structured-chat-zero-shot-react-description",
    verbose=False
)

#agent.run("Tell me about the arts and culture of Berlin")

response=agent.run(input={"pages": "Montreal", "query_str": "What's the arts and culture scene in montreal?"}) ## GPT-3.5

In [54]:
response

"Montreal boasts a vibrant arts and culture scene, earning it the title of 'Canada's Cultural Capital.' The city hosts numerous cultural events and festivals, including the Montreal International Jazz Festival and the Just for Laughs comedy festival. Montreal's downtown area is particularly lively during the summer. The city also has a rich classical arts scene, with venues like Place des Arts showcasing performances by renowned artists. Montreal's architecture blends colonial buildings in Old Montreal with modern landmarks like the Olympic Stadium and the Montreal Metro stations adorned with public artwork by Quebec artists."